# 6장 데이터 로딩과 저장, 파일 형식

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [3]:
!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv")
df

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [4]:
pd.read_csv("examples/ex2.csv")

names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [5]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [6]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [7]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [22]:
result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna()함수 이용
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [9]:
#pd.options.display.max_rows = 10
result = pd.read_csv("examples/ex6.csv")

result = pd.read_csv("examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [12]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [23]:
data = pd.read_csv("examples/ex5.csv")
data1 = data[['a','b','d']]
data1


,a,b,d
0,1,2,4
1,5,6,8
2,9,10,12


In [24]:
data1.to_csv("examples/out.csv")
!cat examples/out.csv

,a,b,d
0,1,2,4
1,5,6,8
2,9,10,12


In [25]:
data.to_csv("examples/out.csv", sep="|") # 구분자 사용
data.to_csv("examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
data.to_csv("examples/out.csv", index=False, header=False)
data.to_csv("examples/out.csv", index=False, columns=["a", "b", "c"])

!cat examples/out.csv


a,b,c
1,2,3.0
5,6,
9,10,11.0


# 2. JSON  데이터
* JSON(JavaScript Object Notation)은 텍스트 데이터를 저장하고 전송하기 위한 일반적인 형식
* 주로 웹에서 데이터를 전송하거나 저장하는 데 사용
* JSON 파일은 일반적으로 텍스트 형식으로 작성되며, 사람과 기계 모두 이해하기 쉽다.
* JSON 파일은 키-값 쌍의 모음으로 구성
  * 키-값 쌍은 객체(object)라고도 하며, 중괄호 {} 로 묶는다.
  * 배열(array)은 대괄호 [] 안에 값의 목록을 나열하여 정의.

## 2.1 JSON 데이터 읽기/저장
    * pandas의 to_json() -> 데이터프레임을 JSON 파일로 저
    * read_json() -> JSON 파일에서 데이터프레임을 읽어오기

### [예제]  pandas의 to_json() 및 read_json() 메서드를 사용하여 데이터프레임을 JSON 파일로 저장하고 읽기

In [27]:
# 1. pandas 라이브러리 임포트

import pandas as pd

# 2. JSON 파일로 저장할 데이터프레임 생성
data = {
    "name": ["John", "Alice", "Bob"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
}
df = pd.DataFrame(data)

# 3. 데이터프레임을 JSON 파일로 저장
df.to_json("examples/data.json")

# JSON 파일을 읽어와 데이터프레임으로 변환
df_read = pd.read_json("examples/data.json")

print("Original DataFrame:")
print(df)

print("\nDataFrame from JSON file:")
print(df_read)


Original DataFrame:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago

DataFrame from JSON file:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago


# 3. HTML 데이터
## 3.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

In [28]:
import pandas as pd
from pandas import Series, DataFrame

#tables = pd.read_html("examples/fdic_failed_bank_list.html") 
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
len(tables)
print(type(tables))
failures = tables[0]
print(type(failures))
failures.head()

<class 'list'>
<class 'pandas.core.frame.DataFrame'>


,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543
3,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.","March 12, 2023",10540
4,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,"March 10, 2023",10539


In [30]:
# 1) 컬럼명 수정하기
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [31]:
# 2) 데이터 변환 : 문자열 -> datetime
close_timestamps = pd.to_datetime(failures["ClosingDate"])
close_timestamps
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts()

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2017      8
2015      8
2023      5
2016      5
2020      4
2004      4
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [34]:
!pip install html5lib  lxml

In [39]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver", encoding="cp949", flavor='html5lib')

# 데이터프레임 확인
for df in df_list:
    print(df)


     0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)
       N       종목명       현재가      전일비     등락률     액면가       시가총액      상장주식수  \
0    NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
1    1.0      삼성전자   82000.0    400.0  -0.49%   100.0  4895222.0  5969783.0   
2    2.0    SK하이닉스  185500.0   2500.0  +1.37%  5000.0  1350444.0   728002.0   
3    3.0  LG에너지솔루션  395500.0    500.0  +0.13%   500.0   925470.0   234000.0   
4    4.0  삼성바이오로직스  816000.0  15000.0  -1.81%  2500.0   580780.0    71174.0   
..   ...       ...       ...      ...     ...     ...        ...        ...   
76  49.0  HD한국조선해양  119700.0   1300.0  +1.10%  5000.0    84715.0    70773.0   
77  50.0      대한항공   22050.0    350.0  +1.61%  500

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [48]:
# 데이터프레임 변경
df = df_list[1]
data = df[df['종목명'].notnull()] # 조건색인
data.head()

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
1,1.0,삼성전자,82000.0,400.0,-0.49%,100.0,4895222.0,5969783.0,55.31,20039134.0,38.48,4.15,NaN
2,2.0,SK하이닉스,185500.0,2500.0,+1.37%,5000.0,1350444.0,728002.0,54.76,2679276.0,-14.82,-15.61,NaN
3,3.0,LG에너지솔루션,395500.0,500.0,+0.13%,500.0,925470.0,234000.0,4.98,99498.0,74.81,6.36,NaN
4,4.0,삼성바이오로직스,816000.0,15000.0,-1.81%,2500.0,580780.0,71174.0,12.02,56042.0,67.71,9.12,NaN
5,5.0,삼성전자우,68000.0,0.0,0.00%,100.0,559563.0,822887.0,75.48,1378501.0,31.91,NaN,NaN


 #### [실습] : 네이버 주식 데이터 가져오기 - Selenium을 활용하여 read_html() 함수

In [51]:
!pip install selenium

In [53]:
from selenium import webdriver
import pandas as pd

# Chrome 브라우저를 열고 웹 페이지로 이동
driver = webdriver.Chrome()
driver.get("https://finance.naver.com/sise/sise_market_sum.naver")

# 페이지가 로드될 때까지 기다림 (5초)
driver.implicitly_wait(5)

# 페이지의 HTML 가져오기
html = driver.page_source

# HTML을 데이터프레임으로 읽기 시도
df_list = pd.read_html(html)

# 데이터프레임 확인
for df in df_list:
    print(df)

# 데이터프레임 수정

# 브라우저 닫기
driver.quit()


C:\Users\USER\AppData\Local\Temp\ipykernel_20216\2566043073.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(html)


     0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)
       N       종목명       현재가      전일비     등락률     액면가       시가총액      상장주식수  \
0    NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
1    1.0      삼성전자   82000.0    400.0  -0.49%   100.0  4895222.0  5969783.0   
2    2.0    SK하이닉스  185500.0   2500.0  +1.37%  5000.0  1350444.0   728002.0   
3    3.0  LG에너지솔루션  395500.0    500.0  +0.13%   500.0   925470.0   234000.0   
4    4.0  삼성바이오로직스  816000.0  15000.0  -1.81%  2500.0   580780.0    71174.0   
..   ...       ...       ...      ...     ...     ...        ...        ...   
76  49.0  HD한국조선해양  119700.0   1300.0  +1.10%  5000.0    84715.0    70773.0   
77  50.0      대한항공   22050.0    350.0  +1.61%  500

In [285]:
# 수정 1
df = df_list[0]
df

,0,1,2,3,4,5
0,거래량,매수호가,거래대금(백만),시가총액(억),영업이익(억),PER(배)
1,시가,매도호가,전일거래량,자산총계(억),영업이익증가율,ROE(%)
2,고가,매수총잔량,외국인비율,부채총계(억),당기순이익(억),ROA(%)
3,저가,매도총잔량,상장주식수(천주),매출액(억),주당순이익(원),PBR(배)
4,NaN,NaN,NaN,매출액증가율,보통주배당금(원),유보율(%)


In [288]:
df = df_list[1]
df.columns
df.head()

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,82500.0,100.0,+0.12%,100.0,4925071.0,5969783.0,55.31,11614947.0,38.71,4.15,NaN
2,2.0,SK하이닉스,185700.0,2700.0,+1.48%,5000.0,1351900.0,728002.0,54.76,1626943.0,-14.84,-15.61,NaN
3,3.0,LG에너지솔루션,394000.0,1000.0,-0.25%,500.0,921960.0,234000.0,4.98,52084.0,74.52,6.36,NaN
4,4.0,삼성바이오로직스,821000.0,10000.0,-1.20%,2500.0,584339.0,71174.0,12.02,29394.0,68.13,9.12,NaN


In [54]:
# 수정 (2) : NaN 값 제거하기 위한 조건색인을 넣기
data = df_list[1]
data['종목명'].notna() # 조건 색인
data[data['종목명'].notna()] #종목명의 값이 NaN이 아니면, True, NaN이면 False


,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
1,1.0,삼성전자,82000.0,400.0,-0.49%,100.0,4895222.0,5969783.0,55.31,20039134.0,38.48,4.15,NaN
2,2.0,SK하이닉스,185500.0,2500.0,+1.37%,5000.0,1350444.0,728002.0,54.76,2679276.0,-14.82,-15.61,NaN
3,3.0,LG에너지솔루션,395500.0,500.0,+0.13%,500.0,925470.0,234000.0,4.98,99498.0,74.81,6.36,NaN
4,4.0,삼성바이오로직스,816000.0,15000.0,-1.81%,2500.0,580780.0,71174.0,12.02,56042.0,67.71,9.12,NaN
5,5.0,삼성전자우,68000.0,0.0,0.00%,100.0,559563.0,822887.0,75.48,1378501.0,31.91,NaN,NaN
9,6.0,현대차,227500.0,5500.0,-2.36%,5000.0,481234.0,211532.0,38.22,878094.0,5.22,13.68,NaN
10,7.0,기아,108700.0,1500.0,-1.36%,5000.0,437022.0,402044.0,40.31,1107331.0,4.99,20.44,NaN
11,8.0,셀트리온,188100.0,3100.0,-1.62%,1000.0,410152.0,218050.0,23.10,667918.0,51.69,5.07,NaN
12,9.0,POSCO홀딩스,418000.0,4000.0,-0.95%,5000.0,353508.0,84571.0,27.30,399867.0,20.82,3.18,NaN
13,10.0,NAVER,194300.0,6900.0,+3.68%,100.0,315560.0,162409.0,45.47,1385197.0,31.44,4.41,NaN


## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용

### [예제] 

In [57]:
# 1. requests 모듈 설치하기
#!pip install requests
# 2. requests 모듈 임포트하기
import requests
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
print(resp.status_code)
# 4. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
# data[0]["title"]
data[0].keys() # 키 정보 출력
# 5. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues.head()

200


,number,title,labels,state
0,58113,ENH: function to concatenate columns with a separator,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg==', 'url': 'https://api.g...",open
1,58110,DEPR: rename `startingMonth` to `starting_month` (argument in BQuarterB...,[],open
2,58108,ENH: Series.map() that works with a dictionary of key-list pairs,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg==', 'url': 'https://api.g...",open
3,58105,Backport PR #58087 on branch 2.2.x (BLD: Build wheels using numpy 2.0rc1),"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNTA=', 'url': 'https://api....",open
4,58104,COMPAT/TST: Ensure that numpy 2 string dtype converts to object array i...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2ODU=', 'url': 'https://api....",open


### [문제] 네이버 데이터 가져오기

In [58]:
import requests

params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
    'orderBy' : 'sim',
    'keyword' : '파이썬'
}

response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

200
https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC




<!DOCTYPE html>
<html lang="ko">
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <base href="/home" />
    <meta name="robots" content="noindex,nofollow"/>
    <meta name="referrer" content="always"/>
    <meta name="format-detection" content="telephone=no">
    <link rel="shortcut icon" type="image/x-icon" href="https://section.blog.naver.com/favicon.ico?3"/>
    




 

	
		<meta property="og:title" content="파이썬 : 네이버 블로그"/>

		
			
			
			<meta property="og:image" content="https://blogimgs.pstatic.net/nblog/mylog/post/og_default_image_160610.png"/>
			
		
        <meta property="og:description" content="당신의 모든 기록을 담는 공간"/>

		
	
	

<meta property="me:feed:serviceId" content="blog" />

    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>네이버 블로그</title>

    <link rel="stylesheet" type="text/css" href="htt

* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 (1)
  * soup.select_one()
  * soup.select()
  * get_text()
  * 개발자 도구 사용

### [문제] 웹 페이지의 제목 가져오기

In [65]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a') 
    for title in titles:
        print(title.get_text())
else : 
    print(response.status_code)



비주얼스튜디오코드 파이썬 질문합니다,
맥북 파이썬 idel
파이썬 모듈 미리 불러오기
파이썬과 자바
IDLE 파이썬 단축키 알려주세요
파이썬강좌 독학할때 뭐보나요??
파이썬이 미래에 도움이 될까요?
파이썬강의 듣는데가 있나요?
파이썬 라이선스 분석
파이썬독학할 때 어떻게 공부를... 


* Selenium을 이용하여 사이트 자동화하기

### [예제]

In [220]:
!pip install selenium

In [303]:
from selenium import webdriver
driver = webdriver.Chrome()
url = "httpss://www.google.com"
driver.get(url)

### [문제]


In [304]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Chrome 브라우저 실행
driver = webdriver.Chrome()

# Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(5)

try:
    # 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q")

    # 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query)

    # Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")

    # 검색 결과의 제목과 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)

finally:
    # 브라우저 창 닫기
    driver.quit()


제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://namu.wiki/w/OpenAI

제목: OpenAI
URL: https://en.wikipedia.org/wiki/OpenAI

제목: 오픈AI - 위키백과, 우리 모두의 백과사전
URL: https://ko.wikipedia.org/wiki/%EC%98%A4%ED%94%88AI

제목: OpenAI
URL: https://www.linkedin.com/company/openai

제목: OpenAI
URL: https://twitter.com/OpenAI

제목: 설명
URL: https://www.google.com/search?q=OpenAI&sca_esv=e2dc8174cd1c459c&source=hp&ei=pGUKZr-TC5in1e8P86KKiAo&iflsig=ANes7DEAAAAAZgpztFiL7HzSE1JsOY_j4Cf_bpy90-K6&udm=&ved=0ahUKEwi_9bGrwqCFAxWYU_UHHXORAqEQ4dUDCA8&uact=5&oq=OpenAI&gs_lp=Egdnd3Mtd2l6IgZPcGVuQUlIRVAAWCBwAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz#

